<a href="https://colab.research.google.com/github/avanninglis/311RequestData/blob/main/311ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install azure-storage-blob
!pip install pyarrow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import datetime
import calendar


In [4]:


# Read Azure Blob Storage connection string from config.json
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Get connection string from config
connection_string = config['connectionString']

# Create BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_name = 'data'
blob_name = '311Data.csv'
download_file_path = '311DataFromBlob.csv'

blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

with open(download_file_path, 'wb') as download_file:
    download_file.write(blob_client.download_blob().readall())

print(f"Blob {blob_name} has been downloaded")


Blob 311Data.csv has been downloaded


In [5]:
df = pd.read_csv('311DataFromBlob.csv')
Data_df = df.copy()
pd.set_option('display.max_columns', None)

Data_df.head(5)

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,address_type,city,facility_type,status,resolution_description,resolution_action_updated_date,community_board,borough,open_data_channel_type,park_facility_name,park_borough,location_type,x_coordinate_state_plane,y_coordinate_state_plane,latitude,longitude,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,:@computed_region_7mpf_4k6g,location.latitude,location.longitude,location.human_address,intersection_street_1,intersection_street_2,landmark,bbl,vehicle_type,closed_date,due_date,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,taxi_pick_up_location,taxi_company_borough
0,60887125,2024-04-17T02:11:06.000,DOT,Department of Transportation,Street Condition,Pothole,11356.0,7 AVENUE,7 AVENUE,122 STREET,123 STREET,BLOCKFACE,QUEENS,NaN,Open,The Department of Transportation referred this...,2024-04-17T02:11:06.000,07 QUEENS,QUEENS,UNKNOWN,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60890163,2024-04-17T01:51:34.000,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN,2024-04-17T02:19:37.000,0 Unspecified,Unspecified,MOBILE,Unspecified,Unspecified,Residential Building/House,1000135.0,214358.0,40.755025,-73.942664,16858.0,39.0,3.0,33.0,72.0,72.0,40.755025,-73.942664,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60883318,2024-04-17T01:51:32.000,NYPD,New York City Police Department,Noise - Residential,Loud Talking,11236.0,1570 EAST 102 STREET,EAST 102 STREET,SEAVIEW AVENUE,SHORE PARKWAY,ADDRESS,BROOKLYN,NaN,In Progress,NaN,2024-04-17T02:48:04.000,18 BROOKLYN,BROOKLYN,PHONE,Unspecified,BROOKLYN,Residential Building/House,1016286.0,170615.0,40.634917,-73.884576,13827.0,5.0,2.0,8.0,42.0,42.0,40.634917,-73.884576,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",SEAVIEW AVENUE,SHORE PARKWAY,EAST 102 STREET,3.083290e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60886001,2024-04-17T01:51:00.000,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,11367.0,67-45 KISSENA BOULEVARD,KISSENA BOULEVARD,65 AVENUE,MELBOURNE AVENUE,ADDRESS,FLUSHING,NaN,In Progress,NaN,NaN,08 QUEENS,QUEENS,MOBILE,Unspecified,QUEENS,Street/Sidewalk,1035587.0,206861.0,40.734313,-73.814760,14509.0,25.0,3.0,24.0,65.0,65.0,40.734313,-73.814760,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",65 AVENUE,MELBOURNE AVENUE,KISSENA BOULEVARD,4.067920e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60886141,2024-04-17T01:50:53.000,DHS,Department of Homeless Services,Homeless Person Assistance,Chronic,10019.0,200 WEST 52 STREET,WEST 52 STREET,7 AVENUE,BROADWAY,ADDRESS,NEW YORK,NaN,Assigned,The Department of Homeless Services has sent a...,2024-04-17T01:52:22.000,05 MANHATTAN,MANHATTAN,MOBILE,Unspecified,MANHATTAN,Street/Sidewalk,989102.0,217036.0,40.762389,-73.982485,12081.0,11.0,4.0,51.0,10.0,10.0,40.762389,-73.982485,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",7 AVENUE,BROADWAY,WEST 52 STREET,1.010230e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# ['unique_key', 'created_date', 'agency', 'agency_name', 'complaint_type',
#        'descriptor', 'incident_zip', 'incident_address', 'street_name',
#        'cross_street_1', 'cross_street_2', 'address_type', 'city',
#        'facility_type', 'status', 'resolution_description',
#        'resolution_action_updated_date', 'community_board', 'borough',
#        'open_data_channel_type', 'park_facility_name', 'park_borough',
#        'location_type', 'x_coordinate_state_plane', 'y_coordinate_state_plane',
#        'latitude', 'longitude', ':@computed_region_efsh_h5xi',
#        ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q',
#        ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih',
#        ':@computed_region_7mpf_4k6g', 'location.latitude',
#        'location.longitude', 'location.human_address', 'intersection_street_1',
#        'intersection_street_2', 'landmark', 'bbl', 'vehicle_type',
#        'closed_date', 'due_date', 'bridge_highway_name',
#        'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
#        'taxi_pick_up_location', 'taxi_company_borough'],
#       dtype='object')


In [7]:
# Removing NaN
Data_df.dropna(subset=['unique_key', 'incident_zip'], inplace=True)
#formatting columns
Data_df['created_date'] = pd.to_datetime(Data_df['created_date'])
Data_df['closed_date'] = pd.to_datetime(Data_df['closed_date'])
Data_df['due_date'] = pd.to_datetime(Data_df['due_date'])
Data_df['resolution_action_updated_date'] = pd.to_datetime(Data_df['resolution_action_updated_date'])
Data_df['incident_zip'] = Data_df['incident_zip'].astype('Int64')

Data_df.drop(columns=['facility_type',
                      'vehicle_type',
                      'bridge_highway_name',
                      'bridge_highway_direction',
                      'location.human_address',
                      'landmark',
                      'bbl',
                      'road_ramp',
                      'bridge_highway_segment',
                      'taxi_pick_up_location',
                      'taxi_company_borough',
                      'location.latitude',
                      'location.longitude',
                      ':@computed_region_efsh_h5xi',
                      'x_coordinate_state_plane',
                      'y_coordinate_state_plane',
                      ':@computed_region_f5dn_yrer',
                      ':@computed_region_yeji_bk3q',
                      ':@computed_region_92fq_4b7q',
                      ':@computed_region_sbqj_enih',
                      ':@computed_region_7mpf_4k6g',
                      'resolution_description'], inplace=True)

pd.set_option('display.max_columns', None) #to see all the columns
Data_df.head()


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,address_type,city,status,resolution_action_updated_date,community_board,borough,open_data_channel_type,park_facility_name,park_borough,location_type,latitude,longitude,intersection_street_1,intersection_street_2,closed_date,due_date
0,60887125,2024-04-17 02:11:06,DOT,Department of Transportation,Street Condition,Pothole,11356,7 AVENUE,7 AVENUE,122 STREET,123 STREET,BLOCKFACE,QUEENS,Open,2024-04-17 02:11:06,07 QUEENS,QUEENS,UNKNOWN,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,60883318,2024-04-17 01:51:32,NYPD,New York City Police Department,Noise - Residential,Loud Talking,11236,1570 EAST 102 STREET,EAST 102 STREET,SEAVIEW AVENUE,SHORE PARKWAY,ADDRESS,BROOKLYN,In Progress,2024-04-17 02:48:04,18 BROOKLYN,BROOKLYN,PHONE,Unspecified,BROOKLYN,Residential Building/House,40.634917,-73.884576,SEAVIEW AVENUE,SHORE PARKWAY,NaT,NaT
3,60886001,2024-04-17 01:51:00,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,11367,67-45 KISSENA BOULEVARD,KISSENA BOULEVARD,65 AVENUE,MELBOURNE AVENUE,ADDRESS,FLUSHING,In Progress,NaT,08 QUEENS,QUEENS,MOBILE,Unspecified,QUEENS,Street/Sidewalk,40.734313,-73.814760,65 AVENUE,MELBOURNE AVENUE,NaT,NaT
4,60886141,2024-04-17 01:50:53,DHS,Department of Homeless Services,Homeless Person Assistance,Chronic,10019,200 WEST 52 STREET,WEST 52 STREET,7 AVENUE,BROADWAY,ADDRESS,NEW YORK,Assigned,2024-04-17 01:52:22,05 MANHATTAN,MANHATTAN,MOBILE,Unspecified,MANHATTAN,Street/Sidewalk,40.762389,-73.982485,7 AVENUE,BROADWAY,NaT,NaT
5,60889980,2024-04-17 01:50:38,NYPD,New York City Police Department,Blocked Driveway,No Access,11379,57-59 78 STREET,78 STREET,QUEENS MIDTOWN EXPRESSWAY,57 ROAD,ADDRESS,MIDDLE VILLAGE,In Progress,NaT,05 QUEENS,QUEENS,ONLINE,Unspecified,QUEENS,Street/Sidewalk,40.727503,-73.883420,QUEENS MIDTOWN EXPRESSWAY,57 ROAD,NaT,NaT


In [8]:


# Getting unique values of 'agency_name' and assign numeric IDs
agency_type_unique = Data_df['agency_name'].unique()
agencyType_mapping = {agency_name: idx for idx, agency_name in enumerate(agency_type_unique, start=1)}

# Create a DataFrame for agency types with their numeric IDs
AgencyType_id = pd.DataFrame(columns=['agencyType_id', 'agency_name'])

# Add data to the DataFrame
AgencyType_id['agency_name'] = agencyType_mapping.keys()
AgencyType_id['agencyType_id'] = AgencyType_id['agency_name'].map(agencyType_mapping)

# Map agency names to their corresponding IDs
Data_df['agencyType_id'] = Data_df['agency_name'].map(AgencyType_id.set_index('agency_name')['agencyType_id'])

# Display the DataFrame with the newly added 'agencyType_id' column
AgencyType_id




,agencyType_id,agency_name
0,1,Department of Transportation
1,2,New York City Police Department
2,3,Department of Homeless Services
3,4,Department of Health and Mental Hygiene
4,5,Department of Sanitation
5,6,Department of Parks and Recreation
6,7,Department of Consumer and Worker Protection
7,8,Department of Housing Preservation and Develop...
8,9,Department of Environmental Protection
9,10,Economic Development Corporation


In [9]:
location_type_unique = Data_df['location_type'].unique()
locationType_mapping = {location_type: idx for idx, location_type in enumerate(location_type_unique, start=1)}

# Create a DataFrame for complaint types with their numeric IDs
LocationType_id = pd.DataFrame(columns=['locationType_id', 'locationType'])

# Add data to the DataFrame
LocationType_id['locationType_id'] = locationType_mapping.values()
LocationType_id['locationType'] = locationType_mapping.keys()
Data_df['locationType_id'] = Data_df['location_type'].map(locationType_mapping)

LocationType_id

,locationType_id,locationType
0,1,NaN
1,2,Residential Building/House
2,3,Street/Sidewalk
3,4,3+ Family Apartment Building
4,5,Lot
5,6,Club/Bar/Restaurant
6,7,Street
7,8,Park/Playground
8,9,Sidewalk
9,10,Store/Commercial


In [10]:

unique_locations = Data_df[['incident_zip', 'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
                            'city', 'borough', 'location_type', 'latitude', 'longitude','intersection_street_1','intersection_street_2']].drop_duplicates()


# Generating location IDs
location_ids = range(1, len(unique_locations) + 1)
# Creating the Location Dimensional Table
Location_id = pd.DataFrame({
    'location_id': location_ids,
    'locationType_id': unique_locations['location_type'].map(locationType_mapping),  # Using locationType_id as the second column
    'incident_zip': unique_locations['incident_zip'],
    'incident_address': unique_locations['incident_address'],
    'street_name': unique_locations['street_name'],
    'cross_street_1': unique_locations['cross_street_1'],
    'cross_street_2': unique_locations['cross_street_2'],
    'city': unique_locations['city'],
    'borough': unique_locations['borough'],
    'latitude': unique_locations['latitude'],
    'longitude': unique_locations['longitude'],
    'intersection_street_1':unique_locations['intersection_street_1'],
    'intersection_street_2':unique_locations['intersection_street_2']})

# Displaying the Location Dimensional Table
Location_id


,location_id,locationType_id,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,city,borough,latitude,longitude,intersection_street_1,intersection_street_2
0,1,1,11356,7 AVENUE,7 AVENUE,122 STREET,123 STREET,QUEENS,QUEENS,NaN,NaN,NaN,NaN
2,2,2,11236,1570 EAST 102 STREET,EAST 102 STREET,SEAVIEW AVENUE,SHORE PARKWAY,BROOKLYN,BROOKLYN,40.634917,-73.884576,SEAVIEW AVENUE,SHORE PARKWAY
3,3,3,11367,67-45 KISSENA BOULEVARD,KISSENA BOULEVARD,65 AVENUE,MELBOURNE AVENUE,FLUSHING,QUEENS,40.734313,-73.814760,65 AVENUE,MELBOURNE AVENUE
4,4,3,10019,200 WEST 52 STREET,WEST 52 STREET,7 AVENUE,BROADWAY,NEW YORK,MANHATTAN,40.762389,-73.982485,7 AVENUE,BROADWAY
5,5,3,11379,57-59 78 STREET,78 STREET,QUEENS MIDTOWN EXPRESSWAY,57 ROAD,MIDDLE VILLAGE,QUEENS,40.727503,-73.883420,QUEENS MIDTOWN EXPRESSWAY,57 ROAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,7444,3,11231,7 COFFEY STREET,COFFEY STREET,OTSEGO STREET,DWIGHT STREET,BROOKLYN,BROOKLYN,40.673830,-74.010617,OTSEGO STREET,DWIGHT STREET
9992,7445,3,11220,372 49 STREET,49 STREET,3 AVENUE,4 AVENUE,BROOKLYN,BROOKLYN,40.647414,-74.012652,3 AVENUE,4 AVENUE
9993,7446,3,10467,45 EAST MOSHOLU PARKWAY NORTH,EAST MOSHOLU PARKWAY NORTH,JEROME AVENUE,KOSSUTH AVENUE,BRONX,BRONX,40.879437,-73.883230,JEROME AVENUE,KOSSUTH AVENUE
9995,7447,6,11236,1663 RALPH AVENUE,RALPH AVENUE,EAST 77 STREET,EAST 76 STREET,BROOKLYN,BROOKLYN,40.636275,-73.919037,EAST 77 STREET,EAST 76 STREET


In [11]:
# Assuming 'Data_df' is your DataFrame containing the data

# # Selecting unique locations based on specific columns
# unique_locations = Data_df[['incident_zip', 'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
#                             'city', 'borough', 'location_type', 'latitude', 'longitude','intersection_street_1','intersection_street_2']].drop_duplicates()

# # Generating location IDs
# location_ids = range(1, len(unique_locations) + 1)

# # Creating the Location Dimensional Table
# Location_id = pd.DataFrame({
#     'location_id': location_ids,
#     'incident_zip': unique_locations['incident_zip'],
#     'incident_address': unique_locations['incident_address'],
#     'street_name': unique_locations['street_name'],
#     'cross_street_1': unique_locations['cross_street_1'],
#     'cross_street_2': unique_locations['cross_street_2'],
#     'city': unique_locations['city'],
#     'borough': unique_locations['borough'],
#     'latitude': unique_locations['latitude'],
#     'longitude': unique_locations['longitude'],
#     'intersection_street_1': unique_locations['intersection_street_1'],
#     'intersection_street_2': unique_locations['intersection_street_2']
# })

# # Mapping location IDs to 'Data_df' based on incident_address
# Data_df = Data_df.merge(Location_id[['location_id', 'incident_address']], on='incident_address', how='left')

# # Displaying the Location Dimensional Table
# Location_id


In [12]:

# Create a dictionary mapping incident addresses to location IDs
address_to_location_id = Location_id.set_index('incident_address')['location_id'].to_dict()

# Map location IDs to Data_df based on incident address
Data_df['location_id'] = Data_df['incident_address'].map(address_to_location_id)
Data_df

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,address_type,city,status,resolution_action_updated_date,community_board,borough,open_data_channel_type,park_facility_name,park_borough,location_type,latitude,longitude,intersection_street_1,intersection_street_2,closed_date,due_date,agencyType_id,locationType_id,location_id
0,60887125,2024-04-17 02:11:06,DOT,Department of Transportation,Street Condition,Pothole,11356,7 AVENUE,7 AVENUE,122 STREET,123 STREET,BLOCKFACE,QUEENS,Open,2024-04-17 02:11:06,07 QUEENS,QUEENS,UNKNOWN,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaT,NaT,1,1,6555
2,60883318,2024-04-17 01:51:32,NYPD,New York City Police Department,Noise - Residential,Loud Talking,11236,1570 EAST 102 STREET,EAST 102 STREET,SEAVIEW AVENUE,SHORE PARKWAY,ADDRESS,BROOKLYN,In Progress,2024-04-17 02:48:04,18 BROOKLYN,BROOKLYN,PHONE,Unspecified,BROOKLYN,Residential Building/House,40.634917,-73.884576,SEAVIEW AVENUE,SHORE PARKWAY,NaT,NaT,2,2,2
3,60886001,2024-04-17 01:51:00,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,11367,67-45 KISSENA BOULEVARD,KISSENA BOULEVARD,65 AVENUE,MELBOURNE AVENUE,ADDRESS,FLUSHING,In Progress,NaT,08 QUEENS,QUEENS,MOBILE,Unspecified,QUEENS,Street/Sidewalk,40.734313,-73.814760,65 AVENUE,MELBOURNE AVENUE,NaT,NaT,2,3,3
4,60886141,2024-04-17 01:50:53,DHS,Department of Homeless Services,Homeless Person Assistance,Chronic,10019,200 WEST 52 STREET,WEST 52 STREET,7 AVENUE,BROADWAY,ADDRESS,NEW YORK,Assigned,2024-04-17 01:52:22,05 MANHATTAN,MANHATTAN,MOBILE,Unspecified,MANHATTAN,Street/Sidewalk,40.762389,-73.982485,7 AVENUE,BROADWAY,NaT,NaT,3,3,4
5,60889980,2024-04-17 01:50:38,NYPD,New York City Police Department,Blocked Driveway,No Access,11379,57-59 78 STREET,78 STREET,QUEENS MIDTOWN EXPRESSWAY,57 ROAD,ADDRESS,MIDDLE VILLAGE,In Progress,NaT,05 QUEENS,QUEENS,ONLINE,Unspecified,QUEENS,Street/Sidewalk,40.727503,-73.883420,QUEENS MIDTOWN EXPRESSWAY,57 ROAD,NaT,NaT,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,60882623,2024-04-16 04:34:42,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,11236,1663 RALPH AVENUE,RALPH AVENUE,EAST 77 STREET,EAST 76 STREET,ADDRESS,BROOKLYN,Closed,2024-04-16 05:24:19,18 BROOKLYN,BROOKLYN,ONLINE,Unspecified,BROOKLYN,Club/Bar/Restaurant,40.636275,-73.919037,EAST 77 STREET,EAST 76 STREET,2024-04-16 05:24:16,NaT,2,6,7447
9996,60890206,2024-04-16 04:32:58,NYPD,New York City Police Department,Illegal Parking,Parking Permit Improper Use,11205,2 PARK AVENUE,PARK AVENUE,NAVY STREET,MONUMENT WALK,ADDRESS,BROOKLYN,Closed,2024-04-16 05:18:51,02 BROOKLYN,BROOKLYN,MOBILE,Unspecified,BROOKLYN,Street/Sidewalk,40.696188,-73.979920,NAVY STREET,MONUMENT WALK,2024-04-16 05:18:48,NaT,2,3,4378
9997,60886084,2024-04-16 04:32:15,NYPD,New York City Police Department,Illegal Parking,Parking Permit Improper Use,11101,5-47 50 AVENUE,50 AVENUE,5 STREET,VERNON BOULEVARD,ADDRESS,LONG ISLAND CITY,Closed,2024-04-16 04:33:58,02 QUEENS,QUEENS,MOBILE,Unspecified,QUEENS,Street/Sidewalk,40.743116,-73.956067,5 STREET,VERNON BOULEVARD,2024-04-16 04:33:53,NaT,2,3,4381
9998,60884514,2024-04-16 04:31:55,NYPD,New York City Police Department,Animal-Abuse,Other (complaint details),11239,200 BETHEL LOOP,BETHEL LOOP,BEND,VANDALIA AVENUE,ADDRESS,BROOKLYN,Closed,2024-04-16 06:15:30,05 BROOKLYN,BROOKLYN,PHONE,Unspecified,BROOKLYN,Residential Building/House,40.652097,-73.881620,BEND,VANDALIA AVENUE,2024-04-16 06:15:25,NaT,2,2,7448


In [13]:
# Getting unique values of 'complaint_type' and assign numeric IDs
complaint_type_unique = Data_df['complaint_type'].unique()
complaintType_mapping = {complaint_type: idx for idx, complaint_type in enumerate(complaint_type_unique, start=1)}

# Create DataFrame for complaint types with their numeric IDs
ComplaintType_id = pd.DataFrame(columns=['complaintType_id', 'complaintType'])
ComplaintType_id['complaintType_id'] = complaintType_mapping.values()
ComplaintType_id['complaintType'] = complaintType_mapping.keys()
# Add data to the DataFrame
Data_df['complaintType_id'] = Data_df['complaint_type'].map(complaintType_mapping)
ComplaintType_id

,complaintType_id,complaintType
0,1,Street Condition
1,2,Noise - Residential
2,3,Illegal Parking
3,4,Homeless Person Assistance
4,5,Blocked Driveway
...,...,...
120,121,Derelict Vehicles
121,122,Wood Pile Remaining
122,123,Animal Facility - No Permit
123,124,Noise - House of Worship


In [14]:
# Getting unique values of 'status' and assign numeric IDs
status_type_unique = Data_df['status'].unique()
statusType_mapping = {status_type: idx for idx, status_type in enumerate(status_type_unique, start=1)}

# Create a DataFrame for complaint types with their numeric IDs
StatusType_id = pd.DataFrame(columns=['statusType_id', 'statusType'])

# Add data to the DataFrame
StatusType_id['statusType_id'] = statusType_mapping.values()
StatusType_id['statusType'] = statusType_mapping.keys()
Data_df['statusType_id'] = Data_df['status'].map(statusType_mapping)
StatusType_id

,statusType_id,statusType
0,1,Open
1,2,In Progress
2,3,Assigned
3,4,Closed
4,5,Started
5,6,Unspecified
6,7,Pending


In [15]:

park_facility = Data_df['park_facility_name'].unique()
park_facility_mapping = {park_facility: idx for idx, park_facility in enumerate(park_facility, start=1)}


Park_facility_id = pd.DataFrame(columns=['park_facility_id', 'park_facility_name'])

# Add data to the DataFrame
Park_facility_id['park_facility_id'] = park_facility_mapping.values()
Park_facility_id['park_facility_name'] = park_facility_mapping.keys()
Data_df['park_facility_id'] = Data_df['park_facility_name'].map(park_facility_mapping )
Park_facility_id

,park_facility_id,park_facility_name
0,1,Unspecified
1,2,Washington Square Park
2,3,NaN
3,4,William Sheridan Playground
4,5,Ferry Point Park
5,6,Bill Brown Playground
6,7,Gorman Playground
7,8,Alley Pond Park
8,9,Franz Sigel Park
9,10,Fort Totten Park


In [16]:
# Getting unique values of 'status' and assign numeric IDs
community_board_unique = Data_df['community_board'].unique()
community_board_mapping = {community_board: idx for idx, community_board in enumerate(community_board_unique, start=1)}

# Create a DataFrame for complaint types with their numeric IDs
Community_board_id = pd.DataFrame(columns=['community_board_id', 'community_board'])

# Add data to the DataFrame
Community_board_id['community_board_id'] = community_board_mapping.values()
Community_board_id['community_board'] = community_board_mapping.keys()
Data_df['community_board_id'] = Data_df['community_board'].map(community_board_mapping)
Community_board_id


,community_board_id,community_board
0,1,07 QUEENS
1,2,18 BROOKLYN
2,3,08 QUEENS
3,4,05 MANHATTAN
4,5,05 QUEENS
...,...,...
70,71,0 Unspecified
71,72,83 QUEENS
72,73,Unspecified STATEN ISLAND
73,74,95 STATEN ISLAND


In [17]:
# Getting unique values of 'status' and assign numeric IDs
channel_unique = Data_df['open_data_channel_type'].unique()
channel_mapping = {open_data_channel_type: idx for idx, open_data_channel_type in enumerate(channel_unique, start=1)}

# Create a DataFrame for complaint types with their numeric IDs
Channel_id = pd.DataFrame(columns=['channel_id', 'channelType'])

# Add data to the DataFrame
Channel_id['channel_id'] = channel_mapping.values()
Channel_id['channelType'] = channel_mapping.keys()

Data_df['channel_id'] = Data_df['open_data_channel_type'].map(channel_mapping)
Channel_id

,channel_id,channelType
0,1,UNKNOWN
1,2,PHONE
2,3,MOBILE
3,4,ONLINE


In [18]:
Data_df['created_date_id'] = pd.to_datetime(Data_df['created_date']).dt.strftime('%Y%m%d%H%M%S')
Data_df['closed_date_id'] = pd.to_datetime(Data_df['closed_date']).dt.strftime('%Y%m%d%H%M%S')
Data_df['due_date_id'] = pd.to_datetime(Data_df['due_date']).dt.strftime('%Y%m%d%H%M%S')
Data_df['updated_date_id'] = pd.to_datetime(Data_df['resolution_action_updated_date']).dt.strftime('%Y%m%d%H%M%S')
Data_df.head(35)


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,address_type,city,status,resolution_action_updated_date,community_board,borough,open_data_channel_type,park_facility_name,park_borough,location_type,latitude,longitude,intersection_street_1,intersection_street_2,closed_date,due_date,agencyType_id,locationType_id,location_id,complaintType_id,statusType_id,park_facility_id,community_board_id,channel_id,created_date_id,closed_date_id,due_date_id,updated_date_id
0,60887125,2024-04-17 02:11:06,DOT,Department of Transportation,Street Condition,Pothole,11356,7 AVENUE,7 AVENUE,122 STREET,123 STREET,BLOCKFACE,QUEENS,Open,2024-04-17 02:11:06,07 QUEENS,QUEENS,UNKNOWN,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaT,NaT,1,1,6555,1,1,1,1,1,20240417021106,NaN,NaN,20240417021106
2,60883318,2024-04-17 01:51:32,NYPD,New York City Police Department,Noise - Residential,Loud Talking,11236,1570 EAST 102 STREET,EAST 102 STREET,SEAVIEW AVENUE,SHORE PARKWAY,ADDRESS,BROOKLYN,In Progress,2024-04-17 02:48:04,18 BROOKLYN,BROOKLYN,PHONE,Unspecified,BROOKLYN,Residential Building/House,40.634917,-73.884576,SEAVIEW AVENUE,SHORE PARKWAY,NaT,NaT,2,2,2,2,2,1,2,2,20240417015132,NaN,NaN,20240417024804
3,60886001,2024-04-17 01:51:00,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,11367,67-45 KISSENA BOULEVARD,KISSENA BOULEVARD,65 AVENUE,MELBOURNE AVENUE,ADDRESS,FLUSHING,In Progress,NaT,08 QUEENS,QUEENS,MOBILE,Unspecified,QUEENS,Street/Sidewalk,40.734313,-73.814760,65 AVENUE,MELBOURNE AVENUE,NaT,NaT,2,3,3,3,2,1,3,3,20240417015100,NaN,NaN,NaN
4,60886141,2024-04-17 01:50:53,DHS,Department of Homeless Services,Homeless Person Assistance,Chronic,10019,200 WEST 52 STREET,WEST 52 STREET,7 AVENUE,BROADWAY,ADDRESS,NEW YORK,Assigned,2024-04-17 01:52:22,05 MANHATTAN,MANHATTAN,MOBILE,Unspecified,MANHATTAN,Street/Sidewalk,40.762389,-73.982485,7 AVENUE,BROADWAY,NaT,NaT,3,3,4,4,3,1,4,3,20240417015053,NaN,NaN,20240417015222
5,60889980,2024-04-17 01:50:38,NYPD,New York City Police Department,Blocked Driveway,No Access,11379,57-59 78 STREET,78 STREET,QUEENS MIDTOWN EXPRESSWAY,57 ROAD,ADDRESS,MIDDLE VILLAGE,In Progress,NaT,05 QUEENS,QUEENS,ONLINE,Unspecified,QUEENS,Street/Sidewalk,40.727503,-73.883420,QUEENS MIDTOWN EXPRESSWAY,57 ROAD,NaT,NaT,2,3,5,5,2,1,5,4,20240417015038,NaN,NaN,NaN
6,60890122,2024-04-17 01:50:05,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,11365,162-05 71 AVENUE,71 AVENUE,162 STREET,163 STREET,ADDRESS,FRESH MEADOWS,In Progress,NaT,08 QUEENS,QUEENS,MOBILE,Unspecified,QUEENS,Street/Sidewalk,40.730433,-73.806956,162 STREET,163 STREET,NaT,NaT,2,3,6,3,2,1,3,3,20240417015005,NaN,NaN,NaN
7,60886037,2024-04-17 01:50:03,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,10009,426 EAST 14 STREET,EAST 14 STREET,1 AVENUE,AVENUE A,ADDRESS,NEW YORK,In Progress,NaT,03 MANHATTAN,MANHATTAN,PHONE,Unspecified,MANHATTAN,Street/Sidewalk,40.730862,-73.981360,1 AVENUE,AVENUE A,NaT,NaT,2,3,7,6,2,1,6,2,20240417015003,NaN,NaN,NaN
8,60887426,2024-04-17 01:47:04,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,10460,1230 EAST TREMONT AVENUE,EAST TREMONT AVENUE,BRONX RIVER PARKWAY,VAN NEST AVENUE,ADDRESS,BRONX,In Progress,NaT,09 BRONX,BRONX,MOBILE,Unspecified,BRONX,Residential Building/House,40.839721,-73.872316,BRONX RIVER PARKWAY,VAN NEST AVENUE,NaT,NaT,2,2,8,2,2,1,7,3,20240417014704,NaN,NaN,NaN
9,60886035,2024-04-17 01:46:39,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,10454,400 BROOK AVENUE,BROOK AVENUE,EAST 143 STREET,EAST 144 STREET,ADDRESS,BRONX,In Progress,2024-04-17 02:38:37,01 BRONX,BRONX,PHONE,Unspecified,BRONX,Residential Building/House,40.811543,-73.916891,EAST 143 STREET,EAST 144 STREET,NaT,NaT,2,2,9,2,2,1,8,2,20240417014639,NaN,NaN,20240417023837
10,60883280,2024-04-17 01:46:21,NYPD,New York City Police Department,Illegal Parking,Blocked Side

In [19]:
mincreated_date = Data_df['created_date'].min()
minclosed_date = Data_df['closed_date'].min()
mindue_date = Data_df['due_date'].min()
minresolution_action_updated_date = Data_df['resolution_action_updated_date'].min()
print("Mincreated_date", mincreated_date,"\n",
      "minclosed_date",minclosed_date,"\n",
      "mindue_date", mindue_date,"\n",
      "minresolution_action_updated", minresolution_action_updated_date)



maxcreated_date = Data_df['created_date'].max()
maxclosed_date = Data_df['closed_date'].max()
maxdue_date = Data_df['due_date'].max()
maxresolution_action_updated_date = Data_df['resolution_action_updated_date'].max()
print("maxcreated_date", maxcreated_date,"\n",
      "maxclosed_date",maxclosed_date,"\n",
      "maxdue_date", maxdue_date,"\n",
      "maxresolution_action_updated", maxresolution_action_updated_date)



Mincreated_date 2024-04-16 04:31:55 
 minclosed_date 2024-03-19 14:34:00 
 mindue_date 2024-05-16 07:41:38 
 minresolution_action_updated 2024-01-03 10:50:08
maxcreated_date 2024-04-17 02:11:06 
 maxclosed_date 2024-04-17 23:55:00 
 maxdue_date 2024-05-17 01:06:24 
 maxresolution_action_updated 2024-04-17 23:55:00


In [ ]:
def week_of_month(dt):
    year = dt.year
    month =dt.month
    day= dt.day

    cal = calendar.monthcalendar(year,month)
    week_number = (day-1)//7+1
    return week_number

start_date = pd.to_datetime('2024-04-15')
end_date =  pd.to_datetime('2024-05-17')

date_dimension = pd.DataFrame({'date':pd.date_range(start_date, end_date, freq='S')})


date_dimension['year_number'] = date_dimension['date'].dt.year
date_dimension['month_number'] = date_dimension['date'].dt.month
date_dimension['monthName'] = date_dimension['date'].dt.strftime('%B')
date_dimension['day_number'] = date_dimension['date'].dt.day
date_dimension['dayName'] = date_dimension['date'].dt.strftime('%A')
date_dimension['hour_number'] = date_dimension['date'].dt.hour
date_dimension['minutes_number'] = date_dimension['date'].dt.minute
date_dimension['seconds_number'] = date_dimension['date'].dt.second
date_dimension['date_iso_format'] = date_dimension['date'].apply(lambda x: x.isoformat())
date_dimension['date_id'] = date_dimension['date'].dt.strftime('%Y%m%d%H%M%S')
date_dimension['weekOfMonth'] = date_dimension['date'].apply(week_of_month)
date_dimension['weekOfYear'] = date_dimension['date'].dt.strftime('%U')



In [ ]:

new_order = ['date_id','year_number','month_number','monthName','day_number','dayName','hour_number','minutes_number','seconds_number','date_iso_format','weekOfMonth','weekOfYear']
date_dimension = date_dimension[new_order]
date_dimension.head(25)

In [ ]:

# Calculate lead times and handle NaN or NaT values
Data_df['created_to_closed_lead_time'] = Data_df['closed_date'].sub(Data_df['created_date']).replace({pd.NaT: np.nan}).abs()
Data_df['due_date_to_closed_lead_time'] = Data_df['closed_date'].sub(Data_df['due_date']).replace({pd.NaT: np.nan}).abs()
Data_df['created_to_updated_lead_time'] = Data_df['created_date'].sub(Data_df['resolution_action_updated_date']).replace({pd.NaT: np.nan}).abs()
Data_df['created_to_due_date_lead_time'] = Data_df['created_date'].sub(Data_df['due_date']).replace({pd.NaT: np.nan}).abs()
Data_df['updated_to_closed_lead_time'] = Data_df['resolution_action_updated_date'].sub(Data_df['closed_date']).replace({pd.NaT: np.nan}).abs()
Data_df['updated_to_due_date_lead_time'] = Data_df['resolution_action_updated_date'].sub(Data_df['due_date']).replace({pd.NaT: np.nan}).abs()


Data_df.head(25)


In [ ]:


# Convert timedelta to seconds
Data_df['created_to_closed_lead_time'] = Data_df['created_to_closed_lead_time'].dt.total_seconds()
Data_df['due_date_to_closed_lead_time'] = Data_df['due_date_to_closed_lead_time'].dt.total_seconds()
Data_df['created_to_updated_lead_time'] = Data_df['created_to_updated_lead_time'].dt.total_seconds()
Data_df['created_to_due_date_lead_time'] = Data_df['created_to_due_date_lead_time'].dt.total_seconds()
Data_df['updated_to_closed_lead_time'] = Data_df['updated_to_closed_lead_time'].dt.total_seconds()
Data_df['updated_to_due_date_lead_time'] = Data_df['updated_to_due_date_lead_time'].dt.total_seconds()


Data_df.head(50)

In [ ]:
DataBefore = Data_df.copy()

new_col_names = {
    "location_id": "location_id",
    "locationType_id": "locationType_id",
    "complaintType_id": "complaintType_id",
    "agencyType_id":"agencyType_id",
    "community_board_id": "community_board_id",
    "park_facility_id":"park_facility_id",
    "channel_id": "channel_id",
    "statusType_id": "statusType_id",
    "created_date_id": "created_date_id",
    "closed_date_id": "closed_date_id",
    "due_date_id": "due_date_id",
    "updated_date_id": "updated_date_id",
    "created_to_closed_lead_time": "created_to_closed_lead_time",
    "due_date_to_closed_lead_time":"due_date_to_closed_lead_time",
    "created_to_updated_lead_time" :"created_to_updated_lead_time",
    "created_to_due_date_lead_time":"created_to_due_date_lead_time",
    "updated_to_closed_lead_time":"updated_to_closed_lead_time",
    "updated_to_due_date_lead_time":"updated_to_due_date_lead_time"

}

Data_df = Data_df.rename(columns=new_col_names)
Data_df["fact_id"] = range(1, len(Data_df) + 1)

new_order = ["fact_id",
             "location_id",
             "locationType_id",
             "complaintType_id",
             "agencyType_id",
             "community_board_id",
             "channel_id",
             "statusType_id",
             "created_date_id",
             "closed_date_id",
             "due_date_id",
             "updated_date_id",
             "created_to_closed_lead_time",
             "due_date_to_closed_lead_time",
             "created_to_updated_lead_time" ,
             "created_to_due_date_lead_time",
             "updated_to_closed_lead_time",
             "updated_to_due_date_lead_time"]

Fact_Table = Data_df[new_order]
Fact_Table.to_csv('Fact_Table.csv', index=False)
# Display the DataFrame
Fact_Table.head(25)






In [ ]:
# Load Azure Storage connection string and container name from config.json
with open('config.json', 'r') as f:
    config = json.load(f)

connection_string = config['connectionString']


# Path to your local CSV file
container_name = 'data'
file_path = "Fact_Table.csv"
blob_name = "311FactTable"

# Create a blob service client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Create a blob client using the container and blob name
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload the file
with open(file_path, "rb") as data:
    blob_client.upload_blob(data)